In [1]:
# default_exp core.library
# export
from ascrum.core.config import config
from pathlib import Path
import yaml
import os
import time
import getpass
import shutil
import time
import mimetypes
import json
from filelock import FileLock

In [2]:
# export
class Library:
    def __init__(self, path: Path):
        self.base = path
        self.archive = config.archive_path if config.archive_path else self.base/'.archive'
        self.archive.mkdir(parents=True, exist_ok=True)
        (self.base/'.meta').mkdir(exist_ok=True)

    def set_document(self, target: str, source_file=None, source_path=None, create_file=False,
                        mimetype=None, mode='wb', user=None):
        'Move a file into the library'

        mimetype = mimetype if mimetype else mimetypes.guess_type(target)[0]
        dest = self.get_path(target)
        dest.parent.mkdir(parents=True, exist_ok=True)
        self._version(dest)

        meta = {
            'mime': mimetype,
            'creator': user,
        } 
        self._set_meta(dest, meta)

        if source_path:
            shutil.move(source_path, dest)
        elif source_file:
            with dest.open(mode) as w:
                shutil.copyfileobj(source_file, w)
        elif create_file:
            return dest.open(mode)

    def get_document(self, path, mode='rb'):
        return self.get_path(path).open(mode)

    def del_document(self, path):
        if path:
            self._version(path)
            self.get_path(path).unlink()

    def mkdir(self, dir):
        p = (self.get_path(dir)/'.meta')
        p.mkdirs(parents=True, exist_ok=True)

    def get_path(self, path):
        while path.startswith('/'): path = path[1:]
        return self.base/path

    def _version(self, path):
        if config.versioning:
            p = self.get_path(path)
            if p.exists():
                ts = time.time()
                target = self.archive/f'{path}.{ts}'
                shutil.move(p.as_posix(), target.as_posix())    

    def _get_meta(self,path):
        p = path.parent/'.meta'/f'{path.name}.meta'        
        try:
            with p.open() as f:
                return json.load(f)
        except:
            return {}
    
    def _set_meta(self, path, meta):
        p = path.parent/'.meta'/f'{path.name}.meta'        
        with p.open('w') as f:
            return json.dump(meta, f)

    def list(self, path='.', pattern='*'):
        def list_item(f):
            m = self._get_meta(f)
            st = f.stat()
            return {
                'name': f.name, 
                'dir': f.is_dir(), 
                'mime': m.get('mime', None),
                'mtime': st.st_mtime,
                'creator': m.get('creator', None),
            }
        p = self.base/path
        return [list_item(f) for f in sorted(p.glob(pattern), 
            key=os.path.getmtime, reverse=True) if not f.name.startswith('.')]


In [3]:
l = Library(Path('/tmp/scrum/library'))


In [84]:
import io
f = io.BytesIO(b"some initial binary data: \x00\x01")
l.set_document('sub/test.txt', source_file=f)


In [6]:
path = Path('./lock.test')
with FileLock(path):
    print(path.stat())
    print('done')

os.stat_result(st_mode=33277, st_ino=5777044, st_dev=66311, st_nlink=1, st_uid=1000, st_gid=1000, st_size=0, st_atime=1604818848, st_mtime=1604818995, st_ctime=1604818995)
done


In [10]:
mimetypes.guess_type('www.docx')[0]

'application/vnd.openxmlformats-officedocument.wordprocessingml.document'

In [56]:
[ f for f in Path('/tmp/scrum/library').iterdir()]

[PosixPath('/tmp/scrum/library/sub'), PosixPath('/tmp/scrum/library/.archive')]